# Noobai-XL-1.1 Text-to-Image Generator

This notebook allows you to use the Noobai-XL-1.1 model with various customization options including:
- Different sampler methods
- Adjustable sampling steps
- Custom seeds
- CFG scale control
- Hi-res fix with 8x-NKMD-Superscale
- Custom image size
- Removed token length limitation

In [ ]:
!pip install diffusers==0.21.4 transformers torch accelerate xformers gradio
!pip install git+https://github.com/huggingface/diffusers.git
!pip install controlnet_aux
!pip install opencv-python

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, EulerAncestralDiscreteScheduler
from diffusers import DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, PNDMScheduler
import gradio as gr
from PIL import Image
import numpy as np
import cv2

In [ ]:
def load_model():
    model_id = "Laxhar/noobai-XL-1.1"
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")
    pipe.enable_xformers_memory_efficient_attention()
    return pipe

pipe = load_model()

In [ ]:
SAMPLERS = {
    "DPM++ 2M": DPMSolverMultistepScheduler,
    "Euler A": EulerAncestralDiscreteScheduler,
    "DDIM": DDIMScheduler,
    "LMS": LMSDiscreteScheduler,
    "Euler": EulerDiscreteScheduler,
    "PNDM": PNDMScheduler
}

def change_sampler(pipe, sampler_name):
    if sampler_name in SAMPLERS:
        pipe.scheduler = SAMPLERS[sampler_name].from_config(pipe.scheduler.config)
    return pipe

In [ ]:
def apply_hires_fix(image, scale_factor, creativity_steps):
    # 8x-NKMD-Superscale implementation
    img_np = np.array(image)
    
    # Convert to YCrCb color space
    img_ycrcb = cv2.cvtColor(img_np, cv2.COLOR_RGB2YCrCb)
    
    # Split channels
    y, cr, cb = cv2.split(img_ycrcb)
    
    # Upscale Y channel
    y_upscaled = cv2.resize(y, None, fx=scale_factor, fy=scale_factor, 
                           interpolation=cv2.INTER_LANCZOS4)
    
    # Apply sharpening based on creativity_steps
    kernel = np.array([[-1,-1,-1],
                      [-1, 9,-1],
                      [-1,-1,-1]]) * (creativity_steps / 100)
    y_upscaled = cv2.filter2D(y_upscaled, -1, kernel)
    
    # Upscale Cr and Cb channels
    cr_upscaled = cv2.resize(cr, (y_upscaled.shape[1], y_upscaled.shape[0]), 
                            interpolation=cv2.INTER_LANCZOS4)
    cb_upscaled = cv2.resize(cb, (y_upscaled.shape[1], y_upscaled.shape[0]), 
                            interpolation=cv2.INTER_LANCZOS4)
    
    # Merge channels
    img_upscaled = cv2.merge([y_upscaled, cr_upscaled, cb_upscaled])
    
    # Convert back to RGB
    img_upscaled = cv2.cvtColor(img_upscaled, cv2.COLOR_YCrCb2RGB)
    
    return Image.fromarray(img_upscaled)

In [ ]:
def generate_image(prompt, negative_prompt, sampler_name, steps, seed, cfg_scale, 
                  width, height, use_hires_fix, hires_scale, creativity_steps):
    
    # Set the seed if provided
    if seed != -1:
        generator = torch.Generator("cuda").manual_seed(seed)
    else:
        generator = torch.Generator("cuda").manual_seed(np.random.randint(0, 2**32 - 1))
    
    # Change sampler
    pipe = change_sampler(pipe, sampler_name)
    
    # Generate image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=steps,
        guidance_scale=cfg_scale,
        width=width,
        height=height,
        generator=generator
    ).images[0]
    
    # Apply hires fix if requested
    if use_hires_fix:
        image = apply_hires_fix(image, hires_scale, creativity_steps)
    
    return image

In [ ]:
def create_interface():
    with gr.Blocks() as demo:
        with gr.Row():
            with gr.Column():
                prompt = gr.Textbox(label="Prompt", lines=3)
                negative_prompt = gr.Textbox(label="Negative Prompt", lines=3)
                sampler = gr.Dropdown(
                    choices=list(SAMPLERS.keys()),
                    value="DPM++ 2M",
                    label="Sampler Method"
                )
                steps = gr.Slider(minimum=1, maximum=150, value=30, step=1, label="Sampling Steps")
                seed = gr.Number(value=-1, label="Seed (-1 for random)")
                cfg = gr.Slider(minimum=1, maximum=20, value=7, step=0.5, label="CFG Scale")
                
                with gr.Row():
                    width = gr.Slider(minimum=256, maximum=2048, value=512, step=64, label="Width")
                    height = gr.Slider(minimum=256, maximum=2048, value=512, step=64, label="Height")
                
                use_hires = gr.Checkbox(label="Use Hires Fix")
                with gr.Group() as hires_group:
                    hires_scale = gr.Slider(minimum=1, maximum=4, value=2, step=0.5, label="Hires Scale")
                    creativity = gr.Slider(minimum=0, maximum=100, value=50, step=5, label="Creativity Steps")
                
                generate_btn = gr.Button("Generate")
            
            with gr.Column():
                output = gr.Image(label="Generated Image")
        
        generate_btn.click(
            fn=generate_image,
            inputs=[
                prompt, negative_prompt, sampler, steps, seed, cfg,
                width, height, use_hires, hires_scale, creativity
            ],
            outputs=output
        )
        
        use_hires.change(
            fn=lambda x: gr.Group.update(visible=x),
            inputs=[use_hires],
            outputs=[hires_group]
        )
    
    return demo

demo = create_interface()
demo.launch(share=True, debug=True)